In [4]:
# Import necessary libraries for the notebook and widgets
import requests
import ipywidgets as widgets
from IPython.display import display, Markdown, clear_output

# Define the function for querying the NIH Reporter API
def query_nih_reporter_by_criteria(org_name=None, department=None, pi_name=None, additional_criteria=None):
    url = 'https://api.reporter.nih.gov/v2/projects/search'
    criteria = {"criteria": {}}

    if org_name:
        criteria["criteria"]["org_names"] = [org_name]
    if department:
        criteria["criteria"]["advanced_text_search"] = {"search_text": department}
    if pi_name:
        criteria["criteria"]["pi_names"] = [{"any_name": pi_name}]
    if additional_criteria:
        for key, value in additional_criteria.items():
            criteria["criteria"][key] = value

    params = {
        "offset": 0,
        "limit": 100 # Adjust as needed
    }
    params.update(criteria)

    response = requests.post(url, json=params)
    
    return response.json().get('results', []) if response.status_code == 200 else []

# Widget for the organization name input
org_widget = widgets.Text(
    value='',
    placeholder='Enter Organization Name',
    description='Organization:',
    disabled=False
)

# Widget for the department input
dept_widget = widgets.Text(
    value='',
    placeholder='Enter Department',
    description='Department:',
    disabled=False
)

# Widget for the PI name input
pi_widget = widgets.Text(
    value='',
    placeholder='Enter PI Name',
    description='PI Name:',
    disabled=False
)

# Button to trigger the search
search_button = widgets.Button(description="Search")

# Output area for displaying the search results
output = widgets.Output()

def on_search_button_clicked(b):
    with output:
        clear_output()
        grants = query_nih_reporter_by_criteria(
            org_name=org_widget.value,
            department=dept_widget.value,
            pi_name=pi_widget.value
        )
        if grants:
            for grant in grants:
                display(Markdown(f"### {grant.get('project_title')}"))
                pis = grant.get('principal_investigators', [])
                pi_names = ', '.join([pi.get('full_name') for pi in pis])
                display(Markdown(f"**Principal Investigators:** {pi_names}"))
                display(Markdown(f"**Organization:** {grant.get('organization', {}).get('org_name')}"))
                display(Markdown(f"**Project Start Date:** {grant.get('project_start_date')}"))
                display(Markdown(f"**Project End Date:** {grant.get('project_end_date')}"))
                display(Markdown("---"))
        else:
            display(Markdown("No results found."))

search_button.on_click(on_search_button_clicked)

# Display widgets
display(org_widget, dept_widget, pi_widget, search_button, output)


Text(value='', description='Organization:', placeholder='Enter Organization Name')

Text(value='', description='Department:', placeholder='Enter Department')

Text(value='', description='PI Name:', placeholder='Enter PI Name')

Button(description='Search', style=ButtonStyle())

Output()